In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
import os,sys
sys.path.insert(0,"../torchxrayvision/")
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import sys
import torch.nn.functional as F
import pandas as pd
import sklearn
import utils

In [10]:
import torchxrayvision as xrv

In [11]:
nn_model = xrv.models.DenseNet(weights="all")#.cuda()
nn_model.op_threshs = None # to prevent sigmoid

In [12]:
import torchvision, torchvision.transforms
transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(224)])

In [13]:
# data is at `git checkout 8e7e5900032e0eacbfee0e604561ea3a258e2b2a`
# https://github.com/ieee8023/covid-chestxray-dataset
path_to_covid_dataset = "../covid-chestxray-dataset"

In [365]:
dmerge = xrv.datasets.COVID19_Dataset(imgpath=path_to_covid_dataset + "/images",
                                      csvpath=path_to_covid_dataset + "/metadata.csv",
                                      transform=transform, views=["PA", "AP"])

In [366]:
dmerge.csv = dmerge.csv.reset_index()

In [16]:
dmerge

{'Aspergillosis': {0.0: 517, 1.0: 1},
 'Aspiration': {0.0: 517, 1.0: 1},
 'Bacterial': {0.0: 472, 1.0: 46},
 'COVID-19': {0.0: 186, 1.0: 332},
 'Chlamydophila': {0.0: 517, 1.0: 1},
 'Fungal': {0.0: 495, 1.0: 23},
 'H1N1': {0.0: 517, 1.0: 1},
 'Herpes ': {0.0: 515, 1.0: 3},
 'Influenza': {0.0: 514, 1.0: 4},
 'Klebsiella': {0.0: 509, 1.0: 9},
 'Legionella': {0.0: 511, 1.0: 7},
 'Lipoid': {0.0: 510, 1.0: 8},
 'MERS-CoV': {0.0: 510, 1.0: 8},
 'MRSA': {0.0: 517, 1.0: 1},
 'Mycoplasma': {0.0: 513, 1.0: 5},
 'No Finding': {0.0: 504, 1.0: 14},
 'Nocardia': {0.0: 514, 1.0: 4},
 'Pneumocystis': {0.0: 496, 1.0: 22},
 'Pneumonia': {0.0: 25, 1.0: 493},
 'SARS': {0.0: 502, 1.0: 16},
 'Staphylococcus': {0.0: 517, 1.0: 1},
 'Streptococcus': {0.0: 501, 1.0: 17},
 'Tuberculosis': {0.0: 507, 1.0: 11},
 'Varicella': {0.0: 513, 1.0: 5},
 'Viral': {0.0: 150, 1.0: 368}}


COVID19_Dataset num_samples=518 views=['PA', 'AP']

In [368]:
dmerge.csv = dmerge.csv.join(pd.DataFrame(dmerge.labels, columns=["gt_" + path for path in dmerge.pathologies]))

In [369]:
scores = pd.read_csv(path_to_covid_dataset + "/annotations/covid-severity-scores.csv", skiprows=5)
scores = scores.set_index("filename")

In [370]:
scores.head()

,geographic_mean,opacity_mean
filename,,
01E392EE-69F9-4E33-BFCE-E5C968654078.jpeg,6.0,4.0
03BF7561-A9BA-4C3C-B8A0-D3E585F73F3C.jpeg,2.7,2.0
1-s2.0-S0140673620303706-fx1_lrg.jpg,2.0,2.0
1-s2.0-S1684118220300608-main.pdf-001.jpg,3.7,3.0
1-s2.0-S1684118220300608-main.pdf-002.jpg,1.7,1.7


In [371]:
dmerge.csv = dmerge.csv.set_index("filename").join(scores, on="filename").reset_index()

In [372]:
dmerge.csv.geographic_mean[~dmerge.csv.geographic_mean.isna()].describe()

count    90.000000
mean      3.280000
std       2.332631
min       0.000000
25%       1.300000
50%       3.000000
75%       4.700000
max       8.000000
Name: geographic_mean, dtype: float64

In [373]:
# dl = torch.utils.data.DataLoader(dmerge, batch_size=32, num_workers=2, pin_memory=True)
# d_feats = []
# d_preds = []
# d_pixels = []
# with torch.no_grad():
#     for i, batch in enumerate(dl):
#         imgs = batch["img"]#.cuda()
#         out = nn_model.features(imgs)
#         out = F.relu(out, inplace=True)
#         out = F.adaptive_avg_pool2d(out, (1, 1))
#         pred = nn_model.classifier(out.view(out.size(0), -1))
#         d_pixels.append(imgs)
#         d_feats.append(out.detach().cpu().numpy())
#         d_preds.append(pred.detach().cpu().numpy())
#         if i %10 == 0:
#             print(i)
# d_feats = np.concatenate(d_feats).reshape(-1,1024)
# d_preds = np.concatenate(d_preds)
# d_pixels = np.concatenate(d_pixels)

In [374]:
import pickle
#pickle.dump([d_feats, d_preds, d_pixels], open("d_feats2.pkl","wb"))
d_feats, d_preds, d_pixels = pickle.load(open("d_feats2.pkl","rb"))

In [375]:
d_feats.shape

(518, 1024)

In [376]:
dmerge.csv = dmerge.csv.join(pd.DataFrame(d_preds, columns=xrv.datasets.default_pathologies))

In [377]:
dmerge.csv["country"] = dmerge.csv.location.str.split(",").str[-1].str.strip()

In [378]:
continents = pd.read_csv("continents.csv.zip")
continents = continents[["name", "region", "sub-region"]]

In [379]:
dmerge.csv = dmerge.csv.join(continents.set_index("name"), on=["country"])

In [380]:
# should be empty (it not then change the country name)
dmerge.csv [(~dmerge.csv.country.isna() & dmerge.csv.region.isna())].country

Series([], Name: country, dtype: object)

In [381]:
d_preds.shape

(518, 18)

In [382]:
d_feats.shape

(518, 1024)

In [383]:
dmerge.csv.country = dmerge.csv.country.fillna("Unknown")
dmerge.csv.region = dmerge.csv.region.fillna("Unknown")
dmerge.csv["sub-region"] = dmerge.csv["sub-region"].fillna("Unknown")

In [384]:
dmerge.csv["Frontal"] = dmerge.csv.view.isin(["AP", "PA", "AP Supine"])
dmerge.csv["Lateral"] = dmerge.csv.view.isin(["L"])
dmerge.csv["SurvivalY"] = dmerge.csv.survival.isin(["Y"])
dmerge.csv["SurvivalN"] = dmerge.csv.survival.isin(["N"])
dmerge.csv["Went ICUY"] = dmerge.csv.went_icu.isin(["Y"])
dmerge.csv["Went ICUN"] = dmerge.csv.went_icu.isin(["N"])
dmerge.csv["IntubationY"] = dmerge.csv.intubated.isin(["Y"])
dmerge.csv["IntubationN"] = dmerge.csv.intubated.isin(["N"])
dmerge.csv["COVID-19"] = dmerge.csv["finding"].str.contains("COVID-19")
dmerge.csv["RT-PCR Positive"] = dmerge.csv["RT_PCR_positive"].isin(["Y"])
dmerge.csv["Male Ratio"] = dmerge.csv.sex == "M"

In [385]:
dmerge.csv.groupby("patientid").first()[["age","Male Ratio"]].mean()

age           54.231618
Male Ratio     0.551320
dtype: float64

In [386]:
r = dmerge.csv.groupby("patientid").first().groupby(["region", "sub-region"]).agg({
                                                         'filename':["count"],
                                                         'age':['mean'],
                                                         'SurvivalY':['sum'],
                                                         'SurvivalN':['sum'],
                                                         'Went ICUY':['sum'],
                                                         'Went ICUN':['sum'],
                                                         'IntubationY':['sum'],
                                                         'IntubationN':['sum'],
                                                         'COVID-19':['sum'],
                                                         'RT-PCR Positive':["sum"],
                                                         'Male Ratio':['mean'],
                                                        }).round(2)
def merge(df,a):
    df[a + " Y/N"] = df[a + 'Y',"sum"].astype("str") + "/" + df[a + 'N',"sum"].astype("str")
    del df[a + 'Y']
    del df[a + 'N']
    
merge(r,"Survival")
merge(r,"Went ICU")
merge(r,"Intubation")
r

filename    age COVID-19  \
                                            count   mean      sum   
region   sub-region                                                 
Africa   Northern Africa                        5  40.00        0   
Americas Latin America and the Caribbean        3  44.67        1   
         Northern America                      21  50.30       11   
Asia     Eastern Asia                          48  54.32       45   
         South-eastern Asia                     9  46.67        8   
         Southern Asia                         14  48.86       13   
         Western Asia                           9  41.00        4   
Europe   Eastern Europe                         2  60.00        0   
         Northern Europe                       30  59.88       21   
         Southern Europe                       99  57.83       67   
         Western Europe                        34  56.40       30   
Oceania  Australia and New Zealand             39  50.65        1   
Unknown  Unknown                               28  54.16       18   

                                         RT-PCR Positive Male Ratio  \
                                                     sum       mean   
region   sub-region                                                   
Africa   Northern Africa                               0       0.00   
Americas Latin America and the Caribbean               1       0.67   
         Northern America                             10       0.57   
Asia     Eastern Asia                                 35       0.31   
         South-eastern Asia                            8       0.56   
         Southern Asia                                13       0.71   
         Western Asia                                  2       0.78   
Europe   Eastern Europe                                0       1.00   
         Northern Europe                              17       0.57   
         Southern Europe                              54       0.61   
         Western Europe                                4       0.59   
Oceania  Australia and New Zealand                     2       0.54   
Unknown  Unknown                                       0       0.61   

                                         Survival Y/N Went ICU Y/N  \
                                                                     
region   sub-region                                                  
Africa   Northern Africa                          0/1          0/0   
Americas Latin America and the Caribbean          1/0          0/1   
         Northern America                         5/4          7/1   
Asia     Eastern Asia                            16/3          3/5   
         South-eastern Asia                       4/1          1/3   
         Southern Asia                            4/0          3/0   
         Western Asia                             2/0          2/0   
Europe   Eastern Europe                           0/0          0/0   
         Northern Europe                          3/3          7/0   
         Southern Europe                         27/2         18/9   
         Western Europe                           8/0         7/23   
Oceania  Australia and New Zealand                4/0          1/1   
Unknown  Unknown                                  3/2          5/0   

                                         Intubation Y/N  
                                                         
region   sub-region                                      
Africa   Northern Africa                            0/0  
Americas Latin America and the Caribbean            0/1  
         Northern America                           7/3  
Asia     Eastern Asia                               5/6  
         South-eastern Asia                         1/3  
         Southern Asia                              3/0  
         Western Asia                               1/0  
Europe   Eastern Europe                             0/0  
         Northern Europe                

In [387]:
#print(r.to_latex())

In [388]:
r = dmerge.csv.groupby(["view"]).agg({
                                                         'filename':["count"],
                                                         'age':['mean'],
                                                         'SurvivalY':['sum'],
                                                         'SurvivalN':['sum'],
                                                         'Went ICUY':['sum'],
                                                         'Went ICUN':['sum'],
                                                         'IntubationY':['sum'],
                                                         'IntubationN':['sum'],
                                                         'COVID-19':['sum'],
                                                         'RT-PCR Positive':["sum"],
                                                         'Male Ratio':['mean'],
                                                        }).round(2)

def merge(df,a):
    df[a + " Y/N"] = df[a + 'Y',"sum"].astype("str") + "/" + df[a + 'N',"sum"].astype("str")
    del df[a + 'Y']
    del df[a + 'N']
    
merge(r,"Survival")
merge(r,"Went ICU")
merge(r,"Intubation")
#r["Survival Y/N"] = r['SurvivalY',"sum"].astype("str") + "/" + r['SurvivalN',"sum"].astype("str")

In [389]:
r

,filename,age,COVID-19,RT-PCR Positive,Male Ratio,Survival Y/N,Went ICU Y/N,Intubation Y/N
,count,mean,sum,sum,mean,,,
view,,,,,,,,
AP,192,57.93,141,109,0.54,36/13,58/6,51/15
PA,326,52.21,191,112,0.54,104/19,53/67,33/55


In [390]:
#print(r.to_latex())

In [391]:
#specify task
task = "survival"

In [336]:
if task=="geographic-extent":
    print(task)
    target_str = "Geographic Extent"
    labels_mask = ~np.isnan(dmerge.csv.geographic_mean.astype(float))
    labels = dmerge.csv.geographic_mean[labels_mask]
    method = "linear"

if task=="opacity":
    print(task)
    target_str = "Opacity"
    labels_mask = ~np.isnan(dmerge.csv.opacity_mean.astype(float))
    labels = dmerge.csv.opacity_mean[labels_mask]
    method = "linear"

In [337]:
if task=="covid19":
    print(task)
    target_str = "COVID-19"
    labels_mask = ~dmerge.csv.finding.str.contains("todo") & \
                  (dmerge.csv.offset.astype("float") > 0) & \
                  (dmerge.csv.offset.astype("float") < 8) 
    labels = dmerge.csv.finding.str.contains("COVID-19")[labels_mask]
    method = "logistic"

In [338]:
if task=="viral-bacterial":
    print(task)
    target_str = "Viral or Bacterial"
    labels_mask = ((dmerge.csv.gt_Bacterial == 1) | (dmerge.csv.gt_Viral == 1))& \
                  (dmerge.csv.offset.astype("float") > 0) & \
                  (dmerge.csv.offset.astype("float") < 8) 
    labels = (dmerge.csv.gt_Viral == 1)[labels_mask]
    method = "logistic"

In [339]:
if task=="survival":
    print(task)
    target_str = "Survival"
    labels_mask = dmerge.csv.survival.isin(["Y","N"]) & \
                  ~(dmerge.csv.intubation_present == "Y") & \
                  (dmerge.csv.offset.astype("float") > 0) & \
                  (dmerge.csv.offset.astype("float") < 8)
    labels = dmerge.csv.survival[labels_mask] == "Y"
    method = "logistic"

survival


In [340]:
if task=="icu-stay":
    print(task)
    target_str = "ICU Stay"
    labels_mask = dmerge.csv.went_icu.isin(["Y","N"]) & \
                  ~(dmerge.csv.in_icu == "Y") & \
                  ~(dmerge.csv.intubation_present == "Y") & \
                  (dmerge.csv.offset.astype("float") > 0) & \
                  (dmerge.csv.offset.astype("float") < 8)
    labels = (dmerge.csv.went_icu[labels_mask] == "Y")[labels_mask]
    method = "logistic"

In [341]:
if task=="intubated":
    print(task)
    target_str = "Intubated"
    labels_mask = dmerge.csv.intubated.isin(["Y","N"]) & \
                  ~(dmerge.csv.intubation_present == "Y") & \
                  (dmerge.csv.offset.astype("float") > 0) & \
                  (dmerge.csv.offset.astype("float") < 8)
    labels = (dmerge.csv.intubated[labels_mask] == "Y")[labels_mask]
    method = "logistic"

In [342]:
labels_mask.sum()

53

In [343]:
dmerge.csv.region[labels_mask].unique()

array(['Asia', 'Americas', 'Europe', 'Oceania', 'Unknown'], dtype=object)

In [344]:
dmerge.csv["sub-region"][labels_mask].unique()

array(['South-eastern Asia', 'Northern America', 'Eastern Asia',
       'Southern Europe', 'Australia and New Zealand', 'Northern Europe',
       'Unknown', 'Western Europe', 'Latin America and the Caribbean',
       'Southern Asia'], dtype=object)

In [345]:
groupby = "region"

In [346]:
test_regions = []
if method == "logistic":
    for region in dmerge.csv[groupby][labels_mask].dropna().unique():
        counts = labels[labels_mask][dmerge.csv[groupby][labels_mask] == region].value_counts()
        print(region, dict(counts))
        if len(counts) > 1 and counts.min() > 0:
            test_regions.append(region)        
if method == "linear":
    for region in dmerge.csv[groupby][labels_mask].dropna().unique():
        if (dmerge.csv[groupby][labels_mask] == region).sum() > 2:
            test_regions.append(region)    

if "Unknown" in test_regions:
    test_regions.remove("Unknown")
test_regions

Asia {True: 20, False: 3}
Americas {True: 4, False: 1}
Europe {True: 19, False: 3}
Oceania {True: 1}
Unknown {True: 2}


['Asia', 'Americas', 'Europe']

In [347]:
results = pd.DataFrame()

In [348]:
data = d_feats[labels_mask]
for test_region in test_regions:
    results = results.append(utils.evaluate(data, labels, "Intermediate features", plot=False, 
                                      regions = dmerge.csv[groupby][labels_mask],
                                      test_region=test_region,
                                      target_str=target_str, 
                                      method=method), ignore_index=True)
    

In [349]:
data = np.zeros((data.shape[0],1))
for test_region in test_regions:
    results = results.append(utils.evaluate(data, labels, "Baseline prevalence", plot=False, 
                                      regions = dmerge.csv[groupby][labels_mask],
                                      test_region=test_region,
                                      target_str=target_str, 
                                      method=method), ignore_index=True)

In [350]:
data = d_preds[labels_mask]
for test_region in test_regions:
    results = results.append(utils.evaluate(data, labels, "18 outputs", plot=False, 
                                      regions = dmerge.csv[groupby][labels_mask],
                                      test_region=test_region,
                                      target_str=target_str, 
                                      method=method), ignore_index=True)

In [351]:
data = d_preds[labels_mask]
data = data[:,[xrv.datasets.default_pathologies.index("Lung Opacity"),
               xrv.datasets.default_pathologies.index("Pneumonia"),
               xrv.datasets.default_pathologies.index("Infiltration"),
               xrv.datasets.default_pathologies.index("Consolidation")]]

for test_region in test_regions:
    results = results.append(utils.evaluate(data, labels, "4 outputs", plot=False, 
                                      regions = dmerge.csv[groupby][labels_mask],
                                      test_region=test_region,
                                      target_str=target_str, 
                                      method=method), ignore_index=True)

In [352]:
data = d_preds[labels_mask]
data = data[:,[xrv.datasets.default_pathologies.index("Lung Opacity")]]
for test_region in test_regions:
    results = results.append(utils.evaluate(data, labels, "lung opacity output", plot=False, 
                                      regions = dmerge.csv[groupby][labels_mask],
                                      test_region=test_region,
                                      target_str=target_str, 
                                      method=method), ignore_index=True)

In [353]:
# data = d_pixels.reshape(d_pixels.shape[0],-1)[labels_mask]
# for test_region in test_regions:
#     for seed in range(3):
#         results = results.append(utils.evaluate(data, labels, "Image pixels", plot=False, 
#                                           regions = dmerge.csv[groupby][labels_mask],
#                                           test_region=test_region,
#                                           target_str=target_str, 
#                                           method=method, 
#                                           seed=seed,
#                                           usemlp=True), ignore_index=True)

In [354]:
results

,# params,# test samples,AUPRC,AUROC,method,name,test_region
0,1024+1,"{True: 20, False: 3}",0.869565,0.500000,logistic,Intermediate features,Asia
1,1024+1,"{False: 1, True: 4}",0.800000,0.500000,logistic,Intermediate features,Americas
2,1024+1,"{True: 19, False: 3}",0.863636,0.500000,logistic,Intermediate features,Europe
3,1+1,"{True: 20, False: 3}",0.869565,0.500000,logistic,Baseline prevalence,Asia
4,1+1,"{False: 1, True: 4}",0.800000,0.500000,logistic,Baseline prevalence,Americas
5,1+1,"{True: 19, False: 3}",0.863636,0.500000,logistic,Baseline prevalence,Europe
6,18+1,"{True: 20, False: 3}",0.858385,0.450000,logistic,18 outputs,Asia
7,18+1,"{False: 1, True: 4}",0.800000,0.500000,logistic,18 outputs,Americas
8,18+1,"{True: 19, False: 3}",0.851435,0.447368,logistic,18 outputs,Europe
9,4+1,"{True: 20, False: 3}",0.869565,0.500000,logistic,4 outputs,Asia


In [355]:
#print(results.round(2).to_latex(index=False))

In [356]:
if method=="linear":
    a = results.groupby(["name", "method"]).agg({
                                 'Correlation':['mean','std'],
                                 'R^2':['mean','std'],
                                 'MAE':['mean','std'],
                                # 'MSE':['mean','std']}
                                }).round(2)
if method=="logistic":
    a = results.groupby(["name", "method"]).agg({
                                 'AUROC':['mean','std'],
                                 'AUPRC':['mean','std']}).round(2)
a

AUROC       AUPRC      
                                mean   std  mean   std
name                  method                          
18 outputs            logistic  0.47  0.03  0.84  0.03
4 outputs             logistic  0.54  0.07  0.85  0.05
Baseline prevalence   logistic  0.50  0.00  0.84  0.04
Intermediate features logistic  0.50  0.00  0.84  0.04
lung opacity output   logistic  0.55  0.08  0.86  0.05

In [357]:
results[" "] = ""

In [358]:
if method=="linear":
    a = results.groupby([" "," ","name", "# params"]).agg({
                             'Correlation':['mean','std'],
                             'R^2':['mean','std'],
                             'MAE':['mean','std']}).round(2).sort_values(("MAE", "mean"))
if method=="logistic":
    a = results.groupby([" "," ","name", "# params"]).agg({
                                 'AUROC':['mean','std'],
                                 'AUPRC':['mean','std']}).round(2).sort_values(("AUROC", "mean"), ascending=False)
    

for c in a.columns.get_level_values(0).unique():
    a[c+"_"] =  a[(c, "mean")].apply("{0:0.2f}".format) + "$\pm$" + a[(c, "std")].apply("{0:0.2f}".format)
    del a[c]
    a[c] = a[c+"_"]
    del a[c+"_"]


In [359]:
a

AUROC          AUPRC
                                                                
    name                  # params                              
    lung opacity output   1+1       0.55$\pm$0.08  0.86$\pm$0.05
    4 outputs             4+1       0.54$\pm$0.07  0.85$\pm$0.05
    Baseline prevalence   1+1       0.50$\pm$0.00  0.84$\pm$0.04
    Intermediate features 1024+1    0.50$\pm$0.00  0.84$\pm$0.04
    18 outputs            18+1      0.47$\pm$0.03  0.84$\pm$0.03

In [360]:
print(", ".join(sorted(test_regions)))

Americas, Asia, Europe


In [361]:
", ".join(sorted(results.test_region.unique()))

'Americas, Asia, Europe'

In [362]:
print(target_str + "\\\\N=" + str(labels_mask.sum()))

Survival\\N=53


In [363]:
print(target_str + "\\\\" + "0<offset<8 days" + 
                   "\\\\" + "True=" + str(labels[labels_mask].value_counts()[True]) + 
                   "\\\\" + "False=" + str(labels[labels_mask].value_counts()[False]))

Survival\\0<offset<8 days\\True=46\\False=7


In [364]:
#print(a.to_latex(escape=False))